In [ ]:
import torch
import pandas as pd
import numpy as np
import pytorch_transformers

folder = 'd:\git\project1\generated'

In [ ]:
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
df_small = pd.read_csv('/content/small_news.csv')
del df_small['Unnamed: 0']
del df_small['Unnamed: 0.1']
df_small

In [ ]:
tokenized = df_small['title'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

In [ ]:
def token_lenght(cell, max):
    if (len(cell) <= max):
        return cell

max = 30
tokens = pd.DataFrame()
tokens['raw'] = tokenized.apply(lambda x: x if len(x) > max)
tokens = tokens.dropna()
#tokens = tokenized.apply(token_lenght, max=30)

tokens

In [ ]:
max_len = 0
for i in tokens['raw']:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

In [ ]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokens['raw']])

In [ ]:
df_padded = pd.DataFrame(padded)
df_padded

In [ ]:
df_union = df_small.merge(df_padded, left_index=True, right_index=True, how='inner')
df_union

In [ ]:
df_union.to_csv(f'{folder}/date_time_tokens_array_short.csv')

In [ ]:
tokens_padded = df_union.iloc[:, 4:]
tokens_padded.to_csv(f'{folder}/tokens_padded.csv')
tokens_padded

In [ ]:
date_time_title_short = df_union.iloc[:, 1:4]
date_time_title_short.to_csv(f'{folder}/date_time_title_short.csv')